In [ ]:
import time
import torch
from PIL import Image,ImageDraw
from myocr.config import MODEL_PATH
from myocr.models.model import ModelZoo
from myocr.predictors.text_detection_predictor import TextDetectionParamConverter
from myocr.predictors.text_recognition_predictor import TextRecognitionParamConverter

orig_image = Image.open("images/flower.png").convert("RGB")

dec = ModelZoo.load_model("onnx", MODEL_PATH+"dbnet++.onnx", "cuda:0" if torch.cuda.is_available() else "cpu")
model = ModelZoo.load_model("onnx", MODEL_PATH+"crnn_lite_lstm.onnx", "cuda:0" if torch.cuda.is_available() else "cpu")

start_time = time.time()
dec_predictor = dec.predictor(TextDetectionParamConverter(dec.device))
detected = dec_predictor.predict(orig_image)
if not detected:
    print("Nothing detected")
else:
    cvt = TextRecognitionParamConverter()
    p = model.predictor(cvt)
    rec = p.predict(detected)
    ent_time = time.time()
    print(f"predict cost: {ent_time-start_time} \nresult:{rec}")
    draw = ImageDraw.Draw(orig_image)
    for box in detected.bounding_boxes: # type: ignore
        draw.polygon(
            [(box.left, box.bottom), (box.left, box.top), (box.right, box.top), (box.right, box.bottom)],
            outline="red",
            width=2,
        )

    display(orig_image)
    if cvt.retain_croped_imgs:
        for img in cvt.croped_imgs:
            display(img)




/home/robby/miniconda3/envs/myocr/lib/python3.11/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:118: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(
INFO:root:Onnx model /home/robby/.MyOCR/models/dbnet++.onnx loaded to cuda:0
INFO:root:Onnx model /home/robby/.MyOCR/models/crnn_lite_lstm.onnx loaded to cuda:0


detected: None
Nothing detected
